In [1]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

In [2]:
from PyCorrelationMatrixManager.correlation_matrix import *

import os

from src.utilities import *
from src.print_utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))



## Operators

Operators for SU(4) baryons are constructed in full at [su4ops](github.com/chrisculver/su4ops) or a repository of Kimmy Cushman.  Here they are translated into the correct objects for the WickContraction library.  Specifically at the moment the below code sets up a list of "Elemental Operators" in different irreps, with a gamma matrix name and spatial coordinate.  

In [3]:
# load definition of the building blocks of the operators.
from src.udud_elementals import *
import numpy as np

#gammas = [('buchoff',0)]
gammas=[]
gammas += np.array([[('A1g',i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('T1g{}'.format(row),i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('Eg{}'.format(row),i) for i in range(2)] for row in range(2)]).reshape(4,2).tolist()

print(gammas)

gammaName = lambda irrep,i : "\\Gamma^{{ ({},{}) }}".format(irrep, i)

aOps=[] # annihilation operators
cOps=[] # creation operators
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','u','u','u'],'[000]',gammaName(g[0],g[1]))]))
    cOps.append(Operator([baryonSource(1,['u','u','u','u'], '[000]',gammaName(g[0],g[1]))]))
    
    #aOps.append(Operator([baryonSink([1,-1,+1,0.5],[['d','u','u','u'],['u','d','u','u'],...],'x_0',gammaName(g[0],g[1]))]))

[['A1g', '0'], ['T1g0', '0'], ['Eg0', '0'], ['Eg0', '1'], ['Eg1', '0'], ['Eg1', '1']]


In [4]:
pprint(aOps[0])

<IPython.core.display.Math object>

In [5]:
pprint(cOps[0])

<IPython.core.display.Math object>

## Wick Contractions

Now for each combination of annihilation and creation operator we compute the wick contractions, and store every diagram in a list.  The example diagram printed out is the full expression with all indices and dependencies for non-momentum projected baryons.

In [25]:
corrMatrix = CorrelationMatrix(cOps[0:2], aOps[0:2], [t for t in range(32)], [t for t in range(32)])
corrMatrix.contract()
corrMatrix.laphify()

Now we convert the above diagram into a LapH version, in terms of a baryon source function and baryon propagator.  These are defined in the "SU4_Correlators" tex file in the LSD overleaf.

In [26]:
allBaryonTensors, allBaryonSinks, allBaryonProps = corrMatrix.get_baryon_tensor_dictionaries()

In [27]:
allBaryonTensors

{'B([000],t_i,\\Gamma^{ (A1g,0) })': 0,
 'B^*([000],t_i,t_f,\\Gamma^{ (A1g,0) })': 1,
 'B^*([000],t_i,t_f,\\Gamma^{ (T1g0,0) })': 2,
 'B([000],t_i,\\Gamma^{ (T1g0,0) })': 3}

In [28]:
allBaryonProps

{'B^*([000],t_i,t_f,\\Gamma^{ (A1g,0) })': 0,
 'B^*([000],t_i,t_f,\\Gamma^{ (T1g0,0) })': 1}

In [29]:
allBaryonSinks

{'B([000],t_i,\\Gamma^{ (A1g,0) })': 0, 'B([000],t_i,\\Gamma^{ (T1g0,0) })': 1}

## Evaluating Diagrams

Now we generate a cpp file which contracts all of the rank 4 baryon tensors into diagrams.  This is to be used with [lapH_build_and_run](https://gitlab.com/Kimmy.Cushman/laph_build_and_run) on lassen.  Specifically the "Output/define_diagrams_gpu.cpp" file needs to be placed in the src folder of the baryons branch of that directory along with the generated(todo) input file.

In [30]:
create_diagram_gpu_file(corrMatrix.get_all_diagrams(), allBaryonTensors, allBaryonSinks, allBaryonProps)    
create_input_file(str(len(corrMatrix.get_all_diagrams())),gammas)
create_diagram_names_file(corrMatrix.get_all_diagrams())

TODO: Need to handle momenta and baryon flavor!


### Testing Loading Diagrams

In [31]:
from PyCorrelationMatrixManager.diagram_data import *

diagramFile=os.path.join(os.path.expanduser("~"),"Code","LQCD","su4workflow","Input","diagrams_335.txt")

In [32]:
data=DiagramData([diagramFile]).diagram_data

key0 = list(data.keys())[0]

In [33]:
expected=corrMatrix.correlators[0].diagrams[0].name()

In [34]:
corrMatrix.load_diagram_values(data)
corrMatrix.compute_correlators()

In [35]:
corrMatrix.correlators[0].values

[(6.024566958887353e-06-6.0681452749007265e-24j),
 (5.5457695993136156e-08+6.684718338203588e-09j),
 (1.0395860637503887e-08+4.146050006426813e-10j),
 (2.6185749329019765e-09+5.31814254864981e-10j),
 (6.69524196276682e-10+5.470491786925824e-11j),
 (1.510570735107534e-10-5.367703018880032e-12j),
 (6.014791592712196e-11-5.144392110681352e-12j),
 (1.052868305516865e-11-2.393500170874694e-12j),
 (8.177949198681038e-12+1.9944440133928716e-12j),
 (1.4171509601188657e-12+1.0968465738122704e-12j),
 (8.089287195876108e-13+5.67904563955958e-13j),
 (9.234499629516255e-14-7.912014151784156e-14j),
 (-3.4389200344606633e-14-2.5672228498255995e-14j),
 (-3.3253286981280317e-14-8.245172302726601e-15j),
 (-8.551636680188339e-14+2.51991244840276e-14j),
 (-8.215791627260032e-14-1.0449080524700173e-14j),
 (-8.588292063123236e-14+1.8242408433235898e-29j),
 (-8.215791627259694e-14+1.0449080524703379e-14j),
 (-8.55163668018807e-14-2.5199124483992576e-14j),
 (-3.3253286981282765e-14+8.245172302682948e-15j),
 (